<a href="https://colab.research.google.com/github/jong9810/TensorFlow-2.0/blob/main/5_5_CNN_Ex3_CIFAR_10_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Ex3 : CIFAR 10 (7 Conv layer, 5 Pooling layer)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import cifar10

tf.__version__

In [ ]:
(x_train, t_train), (x_test, t_test) = cifar10.load_data()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

In [ ]:
# print(t_train[0])

In [ ]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# print(np.random.randint(3, size=7)) # np.random.randint(3, size=7) : 0 ~ 2 까지 숫자를 7개의 원소를 가지는 배열로 반환해줌

In [ ]:
# a = np.array([1, 2, 3])
# b = np.array([4, 5])
# print(np.concatenate( (a, b) ) ) # np.concatenate( (a, b) ) : numpy 배열 a, b가 있을 때 a의 뒤에 b를 이어붙임

In [ ]:
# c = np.arange(5) # np.arange(5) : 각각의 원소가 자신의 인덱스 수인 5개의 원소를 가지는 numpy 배열을 생성
# print(c)
# np.random.shuffle(c) # numpy 배열 c에 대해 모든 원소의 위치를 랜덤으로 재배열
# print(c)

In [ ]:
# CIFAR 10 데이터 보강 (150% 증대)
# 30도 범위에서 회전, 20% 범위에서 기울임, 20% 범위에서 가로, 세로방향 이동, 상하 반전 가능
gen = ImageDataGenerator(rotation_range=30, shear_range=0.2, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)

augment_ratio = 1.5 # 전체 데이터의 150%
augment_size = int(augment_ratio * x_train.shape[0]) # 

# x_train.shape[0] = 50000, auqment_size = 75000
randidx = np.random.randint(x_train.shape[0], size=augment_size) # 0 ~ 49999 까지의 수 중 랜덤으로 75000개 숫자를 반환하여 배열 생성 (중복 가능)

# .copy()함수를 사용하는 이유 : 원본 데이터를 그대로 사용하면 원본 데이터 훼손 가능성이 있기 때문
x_augmented = x_train[randidx].copy()
t_augmented = t_train[randidx].copy()

x_augmented, t_augmented = gen.flow(x_augmented, t_augmented, batch_size=augment_size, shuffle=False).next()

# 기존의 학습 데이터(50000개)에 보강된 데이터(75000개)를 이어 붙임 ->> 총 125000개 학습 데이터
x_train = np.concatenate( (x_train, x_augmented) )
t_train = np.concatenate( (t_train, t_augmented) )

# 보강된 학습 데이터, 정답 데이터를 기존의 데이터와 랜덤하게 섞음
s = np.arange(x_train.shape[0]) # x_train.shape[0] = 125000,  s = [0, 1, 2, ..., 124998, 124999]
np.random.shuffle(s)
# print(s)

# 학습 데이터의 순서를 위에서 정의한 s 배열의 원소의 순서대로 섞어줌
x_train = x_train[s]
t_train = t_train[s]

In [ ]:
cnn = Sequential()

cnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(32,32,3))) # cifar10텐서 (높이, 너비, 채널)
cnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten()) # 3차원 텐서를 1차원 벡터로 변환
cnn.add(Dense(128, activation='relu')) # 은닉층 개념
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation='softmax')) # 출력층

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
cnn.compile(metrics=['accuracy'], loss='sparse_categorical_crossentropy', optimizer=Adam())
cnn.summary()

In [ ]:
hist = cnn.fit(x_train, t_train, validation_data=(x_test, t_test), epochs=250, batch_size=256)

In [ ]:
cnn.evaluate(x_test, t_test)

In [ ]:
plt.plot(hist.history['loss'], label='trian')
plt.plot(hist.history['val_loss'], label='validation')

plt.title('Loss Trend')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.lengend(loc='best')
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label='trian')
plt.plot(hist.history['val_loss'], label='validation')

plt.title('Accuracy Trend')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.grid()
plt.lengend(loc='best')
plt.show()

### CIFAR 10_v1과 결과를 비교해보면
1. v1 (2 Conv layer, 1 Polling layer)
- epoch이 증가함에 따라 오버피팅은 정확도 70% 이후에 계속해서 증가했음

2. v2 (7 Conv layer, 5 Pooling layer)
- epoch이 증가함에 따라 오버피팅은 정확도는 87%로 거의 일정했고 오버피팅은 거의 발생하지 않았다는 것을 알 수 있었음

3. 결론
- v1에 비교해서 v2에서는 데이터 보강과 CNN 모델의 층을 더 복잡하게 구성하였다
- 그 결과 정확도는 증가하였고 오버피팅은 감소하였다. 
- v1 정확도 : 70%, v2 정확도 : 87%
- v1 : 오버피팅이 epoch이 증가할 수록 증가, v2 : 오버피팅이 거의 발생하지 않음

- 하지만 데이터 보강을 해서 데이터의 수가 배로 증가하였고, 층도 더 많았기 때문에 학습하는 데 걸리는 시간도 증가한다는 것을 알았다. (v1 : 1시간 40분 정도, v2 : )

- 따라서 CNN 모델을 구성할 때에는 목표로 하는 정확도와 얼마만큼의 시간을 쓸 수 있는지에 따라 적절하게 층을 구성하는 것이 중요할 것이다.